In [1]:
import os

In [2]:
%pwd

'c:\\Users\\shash\\fetch_assessment\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\shash\\fetch_assessment'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class SplitingDataConfig:
    root_dir: Path
    data_path: Path
    look_back: int
    split:float

In [6]:
from fetch_assessment.constants import *
from fetch_assessment.utils.common import read_yaml, create_directories

Importing constants
Config path: config\config.yaml
Params path: params.yaml


In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_Spliting_Data_config(self) -> SplitingDataConfig:
        config = self.config.spliting_data
        params = self.params.splitingparameters
        create_directories([config.root_dir])

        spliting_data_config = SplitingDataConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            look_back=params.look_back,
            split=params.split
        )

        return spliting_data_config

In [23]:
import os
import numpy as np
import pandas as pd
from fetch_assessment.logging import logger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [38]:
class Spliting_Data:
    def __init__(self,config:SplitingDataConfig):
        self.config = config

    
    def create_dataset(self,data,look_back):
        X, Y = [], []
        for i in range(len(data) - look_back):
            X.append(data[i:(i + look_back)])
            Y.append(data[i + look_back])
        return np.array(X), np.array(Y)

    def spliting_data(self,X,y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        return X_train, X_test, y_train, y_test

    def saving_data(self):
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(pd.read_csv(self.config.data_path).drop("# Date",axis=1))
        X,y=self.create_dataset(scaled_data,self.config.look_back)
        X_train, X_test, y_train, y_test=self.spliting_data(X,y) 
        np.save(os.path.join(self.config.root_dir,"X_train"),X_train)
        np.save(os.path.join(self.config.root_dir,"X_test"),X_test)
        np.save(os.path.join(self.config.root_dir,"y_train"),y_train)
        np.save(os.path.join(self.config.root_dir,"y_test"),y_test)

In [39]:
try:
    config = ConfigurationManager()
    Split_Data_config = config.get_Spliting_Data_config()
    Split_Data = Spliting_Data(config=Split_Data_config)
    Split_Data.saving_data()
except Exception as e:
    raise e

[2023-11-13 11:27:11,938: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-13 11:27:11,946: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-13 11:27:11,946: INFO: common: created directory at: artifacts]
[2023-11-13 11:27:11,950: INFO: common: created directory at: artifacts/data_spliting_data]
